### Theory About Reproducibility
Reproducing or rather producing the quantized model is the deadliest pain the ass I have felt in a while. It was bad.

Keeping the technicalities about SELECT_OPS and Flex Delegate etc aside, just confirm your setup as follow. I have already talked about using virtual environment as kernel in diskloader.ipynb

Final setup
- OS: Ubuntu Server  
- Python: 3.12  
- TensorFlow: 2.18.0 (regular, CPU-only)  
  - Do NOT use `tensorflow[and-cuda]` (conversion fails).  
  - Regular `tensorflow==2.18.0` works fine with Python 3.12.
    ```bash
    pip uninstall -y tensorflow[and-cuda]
    pip install tensorflow==2.18.*
    ````

In [ ]:
# Assuming virtual environment .tens with python 3.12 is being used
# %pip install tensorflow==2.18.*

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model('best_model_345_classes_30000_examples.keras')

#### Converting the Model

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Enable Flex ops (Select TensorFlow Ops)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS  # ← this is the fix
]

# Optional optimizations
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert
tflite_model = converter.convert()

# Save the model
with open('best_model_345_classes_30000_examples.tflite', 'wb') as f:
    f.write(tflite_model)

#### Loading and Using the model

In [ ]:
interpreter = tf.lite.Interpreter(model_path="best_model_345_classes_30000_examples.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
input_details, output_details